In [1]:
import pandas as pd
import numpy as np
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

from logging import getLogger
from recbole.model.general_recommender import BPR
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

import torch
from recbole.data.interaction import Interaction

import os

In [14]:
def train_model(config_dict, k = 10):
    # configurations initialization

    config = Config(model='BPR', dataset='foursquare', config_file_list=['foursquare_general.yaml'], config_dict = config_dict)

    # init random seed
    init_seed(config['seed'], config['reproducibility'])
    # logger initialization
    init_logger(config)
    logger = getLogger()

    # write config info into log
    logger.info(config)

    # dataset creating and filtering
    dataset = create_dataset(config)
    logger.info(dataset)

    # dataset splitting
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # model loading and initialization
    model = BPR(config, train_data.dataset).to(config['device'])
    logger.info(model)

    # trainer loading and initialization
    trainer = Trainer(config, model)

    # model training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, verbose=False)

    # model evaluation
    test_result = trainer.evaluate(test_data)
    print(test_result)

    return model



def get_history(inter):
    visits = []
    set_uid = set(inter['uid:token'])
    for u in set_uid:
        visits.append(inter[inter['uid:token'] == u]['venue_id:token'].values.tolist())
    
    return visits



def predict_k(model, users, visits, locations, k): 
    #make prediction for users

    input_inter = Interaction({
        'uid': torch.tensor(users),
        'venue_id': torch.tensor(visits),
    })

    with torch.no_grad():
        scores = model.full_sort_predict(input_inter).reshape((len(users), -1))

    # get the 10 items with highest scores
    return np.argsort(scores, axis = 1)[:, -k:]



# select one item at random for each user
def random_choice(a):
    # select one item, but then translate it back
    r_c = np.random.choice(a, 1)
    return int(r_c) - 1



def new_atomic_files(interaction):
    try:
        interaction.to_csv('foursquare/foursquare.inter', index=False, sep = '\t')
        print('saved')
    except:
        print('Error saving the interaction file.')




def single_loop(interaction, m, config_dict, users, locations, k):
    # for m iterations, predict the new locations using the same model
    i = 0

    while i < m:
        print('')
        print('--------- iteration number: ', i)
        print('')

        # train the model
        model = train_model(config_dict)

        #get the users' history
        visits = get_history(interaction)

        #recommendation list
        rec_list = predict_k(model, users, visits, locations, k)

        #select one item in the list
        random_item = np.apply_along_axis(random_choice, 1, rec_list)

        ### update the training set
        current_time = max(interaction['timestamp:token'])+1
        new_locations = pd.DataFrame({'uid:token': users, 'venue_id:token':random_item.tolist(), 
                                      'timestamp:token':[current_time]*len(random_item)}, 
                                      columns=['uid:token', 'venue_id:token', 'timestamp:token'])
        
        interaction = pd.concat([interaction, new_locations], axis = 0).reset_index(drop = True)
        interaction.sort_values(by=['uid:token', 'timestamp:token'], inplace=True)

        new_atomic_files(interaction)

        i+=1


In [15]:
k = 10
config_dict = {
    #environment settings
    'seed': 1234,
    'reproducibility': True,
    'data_path': os.getcwd(), 
    'topk': k
}

m = 3
inter = pd.read_csv('inter.csv')

unique_users = list(set(inter['uid:token']))
unique_locations = list(set(inter['venue_id:token']))

single_loop(inter, 1, config_dict, unique_users, unique_locations, k)


--------- iteration number:  0



17 May 08:31    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 1234
state = INFO
reproducibility = True
data_path = /Users/andreafrasson/Desktop/tesi/Feedback-Loop-for-POI-RS/foursquare
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 4096
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096

OrderedDict([('recall@10', 0.398), ('mrr@10', 0.3578), ('ndcg@10', 0.3673), ('hit@10', 0.398), ('precision@10', 0.0398)])


/var/folders/8_/mkx2sfs13pqgtr19k0_pc89h0000gn/T/ipykernel_1530/361195557.py:68: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(np.random.choice(a, 1))


saved


In [11]:
inter

,uid:token,venue_id:token,timestamp:token
0,1,33236,0
1,1,791,1
2,1,1511,2
3,1,6720,3
4,1,255,4
...,...,...,...
108295,1083,623,95
108296,1083,1313,96
108297,1083,4567,97
108298,1083,4127,98


In [3]:
import os
os.getcwd()

'/Users/andreafrasson/Desktop/tesi/Feedback-Loop-for-POI-RS'